# Validate Quality Metrics of ML (& Markov) Prediction

1️⃣ Error Metrics (Ground Truth vs ML Predicted)

- RMSE
- MAE
- R²
- MAPE
- Distribution RMSE

2️⃣ Kinematic Validity

- Apakah distribusi speed ML mirip real?
- Apakah acceleration patternnya mirip real?

3️⃣ Emission-driven Metric (optional)

- VSP RMSE → apakah kinematic → emission-related behaviour mirip?


4️⃣ Pass/Fail Thresholds

Untuk CI/CD (Airflow), agar dapat:

- max_RMSE
- max_accel_diff
- max_vsp_rmse
- max_speed_bias

In [ ]:
## ================================================================
# 0. PARAMETERS (Papermill will override these)
#    [TAG: parameters]
# ================================================================
INPUT_TEST_DATA = "s3://ml-quality/test_data.pkl"
INPUT_PREDICTED_DATA = "s3://ml-quality/pred_speed_accel.csv"
OUTPUT_METRICS_PATH = "s3://ml-quality/metrics/ml_quality.json"
OUTPUT_PLOT_PATH = "s3://ml-quality/plots/ml_quality.png"

MINIO_ENDPOINT = "http://minio:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password123"

# Quality thresholds (tuning knobs)
MAX_SPEED_RMSE = 3.0        # km/h
MAX_ACCEL_RMSE = 0.5        # m/s^2
MAX_VSP_RMSE = 0.4          # kW/ton
MAX_SPEED_KL = 0.30         # speed distribution KL
MAX_VSP_KL = 0.40           # VSP distribution KL


In [ ]:
# ================================================================
# 1. IMPORTS & MINIO SETUP
# ================================================================
import s3fs
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import io

from sklearn.metrics import r2_score

# Configure S3/MinIO filesystem
fs = s3fs.S3FileSystem(
    key=MINIO_ACCESS_KEY,
    secret=MINIO_SECRET_KEY,
    client_kwargs={'endpoint_url': MINIO_ENDPOINT}
)

print("✅ MinIO filesystem initialized.")


In [ ]:
# ================================================================
# 2. LOAD REAL TEST DATA & ML PREDICTIONS
# ================================================================
print("📥 Loading real test data from:", INPUT_TEST_DATA)
with fs.open(INPUT_TEST_DATA, 'rb') as f:
    df_real = pd.read_pickle(f)

print("📥 Loading ML predictions from:", INPUT_PREDICTED_DATA)
with fs.open(INPUT_PREDICTED_DATA, 'rb') as f:
    df_pred = pd.read_csv(f)

# Align lengths
n = min(len(df_real), len(df_pred))
if n == 0:
    raise ValueError("No overlapping rows between real and predicted data (n=0).")

In [ ]:
df_real = df_real.iloc[:n].reset_index(drop=True)
df_pred = df_pred.iloc[:n].reset_index(drop=True)

# Real data columns
real_speed = df_real["speed_kmh"].values        # km/h
real_acc = df_real["acc_m_s2"].values           # m/s^2

# Predicted data: speed from ML, accel derived from speed
pred_speed = df_pred["speed_kmh"].values        # km/h
pred_acc = np.gradient(pred_speed / 3.6)        # m/s^2 from speed

In [ ]:
# ================================================================
# 3. BASIC METRICS (RMSE / MAE / R2)
# ================================================================
def rmse(a, b):
    return float(np.sqrt(((a - b) ** 2).mean()))

def mae(a, b):
    return float(np.mean(np.abs(a - b)))

# Speed metrics
speed_rmse = rmse(real_speed, pred_speed)
speed_mae = mae(real_speed, pred_speed)
r2_speed = float(r2_score(real_speed, pred_speed))

# Acceleration metrics
acc_rmse = rmse(real_acc, pred_acc)
r2_accel = float(r2_score(real_acc, pred_acc))

print("=== BASIC METRICS ===")
print(f"Speed RMSE       : {speed_rmse:.4f} km/h")
print(f"Speed MAE        : {speed_mae:.4f} km/h")
print(f"Speed R²         : {r2_speed:.4f}")
print(f"Accel RMSE       : {acc_rmse:.4f} m/s²")
print(f"Accel R²         : {r2_accel:.4f}")

In [ ]:
# ================================================================
# 4. VSP COMPUTATION (REAL vs PRED)
# ================================================================
def calc_vsp(speed_ms, acc_ms2):
    """
    MOVESTAR-style VSP for motorcycle (veh_type=1).
    Using coefficients:
      A=0.0251, B=0, C=0.000315, M=0.285, f=0.285
    """
    A, B, C, M, f = 0.0251, 0.0, 0.000315, 0.285, 0.285
    return (A * speed_ms + B * speed_ms**2 + C * speed_ms**3 + M * speed_ms * acc_ms2) / f

vsp_real = calc_vsp(real_speed / 3.6, real_acc)
vsp_pred = calc_vsp(pred_speed / 3.6, pred_acc)

vsp_rmse = rmse(vsp_real, vsp_pred)
print(f"VSP RMSE         : {vsp_rmse:.4f} kW/ton")

In [ ]:
# ================================================================
# 5. DISTRIBUTION METRICS (Speed & VSP)
# ================================================================
def get_distribution(data, bins):
    """Return normalized histogram (probability distribution)."""
    counts, _ = np.histogram(data, bins=bins)
    total = np.sum(counts)
    return counts / total if total > 0 else np.zeros(len(counts))

def kl_divergence(p, q, eps=1e-10):
    """
    KL(P || Q) with small eps to avoid log(0).
    p: model / predicted distribution
    q: real / ground truth distribution
    """
    p = p + eps
    q = q + eps
    p = p / np.sum(p)
    q = q / np.sum(q)
    return float(np.sum(p * np.log(p / q)))

# --- Speed distribution ---
bins_speed = np.arange(0, 120, 5)  # 0–115 km/h
dist_real_speed = get_distribution(real_speed, bins_speed)
dist_pred_speed = get_distribution(pred_speed, bins_speed)

speed_dist_rmse = rmse(dist_real_speed, dist_pred_speed)
speed_kl = kl_divergence(dist_pred_speed, dist_real_speed)

print("\n=== SPEED DISTRIBUTION METRICS ===")
print(f"Speed dist RMSE  : {speed_dist_rmse:.6f}")
print(f"Speed KL(P||Q)   : {speed_kl:.6f}")

# --- VSP distribution ---
bins_vsp = np.arange(-10, 40, 2)  # typical VSP range
dist_real_vsp = get_distribution(vsp_real, bins_vsp)
dist_pred_vsp = get_distribution(vsp_pred, bins_vsp)

vsp_dist_rmse = rmse(dist_real_vsp, dist_pred_vsp)
vsp_kl = kl_divergence(dist_pred_vsp, dist_real_vsp)

print("\n=== VSP DISTRIBUTION METRICS ===")
print(f"VSP dist RMSE    : {vsp_dist_rmse:.6f}")
print(f"VSP KL(P||Q)     : {vsp_kl:.6f}")

In [ ]:
# ================================================================
# 6. CORRELATION METRICS (OPTIONAL BUT NICE)
# ================================================================
corr_speed = float(np.corrcoef(real_speed, pred_speed)[0, 1])
corr_acc = float(np.corrcoef(real_acc, pred_acc)[0, 1])

print("\n=== CORRELATIONS ===")
print(f"Speed Corr       : {corr_speed:.4f}")
print(f"Accel Corr       : {corr_acc:.4f}")

In [ ]:
# ================================================================
# 7. PLOTTING (Time Series + Distributions)
# ================================================================
plt.figure(figsize=(16, 10))

# 1. Speed time series
plt.subplot(2, 2, 1)
plt.plot(real_speed, label="Real speed", alpha=0.7)
plt.plot(pred_speed, label="ML speed", alpha=0.7)
plt.title("Speed vs Time")
plt.xlabel("Time (s)")
plt.ylabel("Speed (km/h)")
plt.legend()

# 2. VSP time series
plt.subplot(2, 2, 2)
plt.plot(vsp_real, label="Real VSP", alpha=0.7)
plt.plot(vsp_pred, label="ML VSP", alpha=0.7)
plt.title("VSP vs Time")
plt.xlabel("Time (s)")
plt.ylabel("VSP (kW/ton)")
plt.legend()

# 3. Speed distribution
plt.subplot(2, 2, 3)
centers_speed = (bins_speed[:-1] + bins_speed[1:]) / 2
plt.bar(centers_speed, dist_real_speed, width=4, alpha=0.5, label="Real", edgecolor="k")
plt.plot(centers_speed, dist_pred_speed, label="ML", linewidth=2)
plt.title(f"Speed Distribution\nKL={speed_kl:.3f}, RMSE={speed_dist_rmse:.3f}")
plt.xlabel("Speed (km/h)")
plt.ylabel("Probability")
plt.legend()

# 4. VSP distribution
plt.subplot(2, 2, 4)
centers_vsp = (bins_vsp[:-1] + bins_vsp[1:]) / 2
plt.bar(centers_vsp, dist_real_vsp, width=1.5, alpha=0.5, label="Real", edgecolor="k")
plt.plot(centers_vsp, dist_pred_vsp, label="ML", linewidth=2)
plt.title(f"VSP Distribution\nKL={vsp_kl:.3f}, RMSE={vsp_dist_rmse:.3f}")
plt.xlabel("VSP (kW/ton)")
plt.ylabel("Probability")
plt.legend()

plt.tight_layout()

print("\n💾 Saving plots to:", OUTPUT_PLOT_PATH)
with fs.open(OUTPUT_PLOT_PATH, 'wb') as f:
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=150)
    buf.seek(0)
    f.write(buf.getvalue())
print("✅ Plot saved.")


In [ ]:
# ================================================================
# 8. SAVE METRICS + PASS / FAIL
# ================================================================
metrics = {
    "sample_size": int(n),

    # Basic regression metrics
    "speed_rmse": speed_rmse,
    "speed_mae": speed_mae,
    "speed_r2": r2_speed,
    "acc_rmse": acc_rmse,
    "acc_r2": r2_accel,

    # VSP metrics
    "vsp_rmse": vsp_rmse,

    # Distribution metrics
    "speed_dist_rmse": speed_dist_rmse,
    "speed_kl": speed_kl,
    "vsp_dist_rmse": vsp_dist_rmse,
    "vsp_kl": vsp_kl,

    # Correlations (optional, for reporting)
    "corr_speed": corr_speed,
    "corr_acc": corr_acc,

    # Thresholds
    "thresholds": {
        "MAX_SPEED_RMSE": MAX_SPEED_RMSE,
        "MAX_ACCEL_RMSE": MAX_ACCEL_RMSE,
        "MAX_VSP_RMSE": MAX_VSP_RMSE,
        "MAX_SPEED_KL": MAX_SPEED_KL,
        "MAX_VSP_KL": MAX_VSP_KL,
    }
}

# Quality gate
passed = (
    (speed_rmse < MAX_SPEED_RMSE) and
    (acc_rmse < MAX_ACCEL_RMSE) and
    (vsp_rmse < MAX_VSP_RMSE) and
    (speed_kl < MAX_SPEED_KL) and
    (vsp_kl < MAX_VSP_KL)
)

metrics["passed"] = bool(passed)

print("\n=== FINAL EVALUATION ===")
print(json.dumps(metrics, indent=2))

print("\n💾 Saving metrics to:", OUTPUT_METRICS_PATH)
with fs.open(OUTPUT_METRICS_PATH, 'w') as f:
    json.dump(metrics, f, indent=4)
print("✅ Metrics saved.")

if passed:
    print("\n✅ ML Quality Validation PASSED (within thresholds).")
else:
    print("\n⚠️ ML Quality Validation FAILED (one or more metrics exceed thresholds).")